In [1]:
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map, process_map

In [2]:
%config Completer.use_jedi = False

In [3]:
%load_ext autoreload
%autoreload 2

In [93]:
pd.options.display.max_rows = 100

In [4]:
import tokenize_code, glob

In [ ]:
path = 'Lib/'
tokens = []
files = glob.glob(path+'/**/*.py', recursive=True)

def do(file):
    global tokens
    try:        
        out = tokenize_code.tokenize_r(file)
        tokens += out
    except Exception as e:
        print(e, file)

x = thread_map(do, files)


  0%|          | 0/952 [00:00<?, ?it/s]

unindent does not match any outer indentation level (<tokenize>, line 107)
'NoneType' object is not iterable Lib\cmd.py
unindent does not match any outer indentation level (<tokenize>, line 100)
'NoneType' object is not iterable Lib\filecmp.py
unindent does not match any outer indentation level (<tokenize>, line 153)
'NoneType' object is not iterable Lib\fractions.py
unindent does not match any outer indentation level (<tokenize>, line 31)
'NoneType' object is not iterable Lib\difflib.py


In [7]:
len(tokens)

1847674

In [6]:
from collections import Counter
import pickle 

In [12]:
with open('lib_tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)

In [14]:
counter_lib = Counter(tokens)

In [15]:
lib_set = set(tokens)

In [17]:
len(lib_set)

63007

In [7]:
with open('lib_tokens.pkl', 'rb') as f:
    tokens = pickle.load( f)

In [265]:
import re
import pandas as pd
datasets = [[]]
file_name = 'python_text.py'

with open(file_name, 'r', encoding='utf-8') as f:
  #my_dict = {"description":[],"code":[]}
  for line in f:
    if line.startswith('#'):
      comment = line.split('\n#')
      if datasets[-1] != []:
        # we are in a new block
        datasets.append(comment)
    else:
      stripped_line = line#.strip()
      if stripped_line:
        datasets[-1].append(stripped_line)


In [266]:
raw_data = {'Description' : [re.sub(r"^#(\d)*",'',x[0]).strip() for x in datasets], 'Code': [''.join(x[1:]) for x in datasets]}
df = pd.DataFrame(raw_data, columns=["Description", "Code"])

In [267]:
df = df.drop(0)

In [268]:
df

,Description,Code
1,write a python function to add two user provid...,"\n\ndef add_two_numbers(num1, num2):\n sum ..."
2,write a program to find and print the largest ...,\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 ...
3,write a program to find and print the smallest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <=...
4,Write a python function to merge two given lis...,"\n\ndef merge_lists(l1, l2):\n return l1 + ..."
5,Write a program to check whether a number is p...,num = 337\n\n\nif num > 1:\n for i in range(...
...,...,...
4366,Write a program to print bit wise OR of two nu...,"a=60\nb=13\n\n\nc=a | b\nprint(""OR"", c)\n\n\n\n\n"
4367,Write a program to print bit wise XOR of two n...,"a=60\nb=13\n\n\nc=a ^ b\nprint(""XOR"", c)\n\n\n..."
4368,Write a program to calculate Binary Ones Compl...,"a=60\n\n\nc=~a\nprint(""Binary Ones Complement""..."
4369,write a program to Binary Left Shift a number,"c=a << 2\nprint(""Binary Left Shift"", c)\n\n\n\n\n"


In [269]:
from tokenize_code import tokenize_r

In [270]:
a = df.iloc[1]['Code']
a

"\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n"

In [271]:
df_final1 = pd.DataFrame()
for _, row in tqdm(df.iterrows()):
    code = tokenize_r(code=row['Code'])
    
    if code:
        row['Code'] = ''.join(a for a in code)
        df_final1 = df_final1.append(row)

0it [00:00, ?it/s]

unexpected EOF while parsing (python_minifier.minify source, line 5) None
unexpected EOF while parsing (python_minifier.minify source, line 3) None
invalid syntax (python_minifier.minify source, line 1) None
invalid syntax (python_minifier.minify source, line 1) None
invalid syntax (python_minifier.minify source, line 2) None
invalid syntax (python_minifier.minify source, line 1) None
unindent does not match any outer indentation level (python_minifier.minify source, line 11) None
unindent does not match any outer indentation level (python_minifier.minify source, line 12) None
invalid syntax (python_minifier.minify source, line 1) None
invalid syntax (python_minifier.minify source, line 6) None
invalid syntax (python_minifier.minify source, line 25) None
unexpected indent (python_minifier.minify source, line 1) None
unexpected EOF while parsing (python_minifier.minify source, line 3) None
unexpected indent (python_minifier.minify source, line 2) None
invalid syntax (python_minifier.min

In [272]:
df_final1.columns

Index(['Code', 'Description'], dtype='object')

In [273]:
df_final1=df_final1[['Description', 'Code']]

In [274]:
df_final1

,Description,Code
1,write a python function to add two user provid...,"\n\ndef add_two_numbers(num1, num2):\n\tsum = ..."
2,write a program to find and print the largest ...,\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 ...
3,write a program to find and print the smallest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <=...
4,Write a python function to merge two given lis...,"\n\ndef merge_lists(l1, l2):\n\treturn l1 + l2\n"
5,Write a program to check whether a number is p...,num = 337\n\n\nif num > 1:\n\tfor i in range(2...
...,...,...
4366,Write a program to print bit wise OR of two nu...,"a = 60\nb = 13\n\n\nc = a | b\nprint(""OR"", c)\n"
4367,Write a program to print bit wise XOR of two n...,"a = 60\nb = 13\n\n\nc = a ^ b\nprint(""XOR"", c)\n"
4368,Write a program to calculate Binary Ones Compl...,"a = 60\n\n\nc = ~a\nprint(""Binary Ones Complem..."
4369,write a program to Binary Left Shift a number,"c = a << 2\nprint(""Binary Left Shift"", c)\n"


In [275]:
df_final1[df_final1['Description'].str.contains('function')]

,Description,Code
1,write a python function to add two user provid...,"\n\ndef add_two_numbers(num1, num2):\n\tsum = ..."
4,Write a python function to merge two given lis...,"\n\ndef merge_lists(l1, l2):\n\treturn l1 + l2\n"
6,Write a python function that prints the factor...,"\n\ndef print_factors(x):\n\tprint(f""The facto..."
8,Write a python function to print whether a num...,\n\ndef check_pnz(num):\n\tif num > 0:\n ...
10,"Write a python function to print powers of 2, ...",\n\ndef two_power(terms):\n\tresult = list(map...
...,...,...
4279,Define a function which can generate and print...,def printTuple():\n\tli = list()\n\tfor i in r...
4283,Write a program which can filter even numbers ...,"li = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nevenNumb..."
4292,Define a class named Shape and its subclass Sq...,class Shape(object):\n\tdef __init__(self):\n ...
4309,write a binary search function which searches ...,"import math\n\n\ndef bin_search(li, element):\..."


In [276]:
df_final1['len'] = df_final1['Code'].str.len()
df_final1 = df_final1.drop(df_final1[df_final1['len'] == 0].index)
df_final1 = df_final1[df_final1['len'] < 251]

In [277]:
df_final1.to_pickle('final_code_samples.pkl', protocol=4)
df_final1.to_csv('final_code_samples.csv')

In [278]:
# Dividing the data into train and validation dataset

train_df = df_final1.sample(frac = 0.85) 
  
# Creating dataframe with rest of the 20% values 
valid_df = df_final1.drop(train_df.index)


In [279]:
train_df

,Description,Code,len
1558,Python program that accepts an integer (n) and...,"\n\ndef custom(n):\n\ta = n\n\tn1 = int(""%s"" %...",144
2447,Write a python program to print positive Numbe...,"\nlist1 = [11, -21, 0, 45, 66, -93]\n\nfor num...",97
2876,Write a function to calculate the electrostati...,"def cal_electrostatic_force(q1, q2, d):\n\tk =...",88
2767,Write a python program for print all elements ...,"\ntest_list = [56, 72, 875, 9, 173]\n\nK = 7\n...",144
291,write a python function using list comprehensi...,def find_evennumbers(input_list):\n\tlist_usin...,121
...,...,...,...
100,write a python program to print a matrix as ou...,"\n\nmatrix = [[1, 2, 3, 4],\n [4, 5, ...",157
2060,Write a function to return the volume of a sphere,def cal_sphere_volume(radius: float) -> float:...,93
3647,32 print current version of python,import sys\nprint(sys.version)\n,30
2038,Write a function to return the area of a square,def cal_area_square(side):\n\treturn side**2\n,43


In [280]:
valid_df

,Description,Code,len
8,Write a python function to print whether a num...,\n\ndef check_pnz(num):\n\tif num > 0:\n ...,148
11,Write a program to filter the numbers in a lis...,"my_list = [11, 45, 74, 89, 132, 239, 721, 21]\...",165
25,Write a function that returns the sum of digit...,\n\ndef digisum(num):\n\tsum_ = 0\n\twhile num...,121
31,Write a function to find the perimeter of a re...,"\n\ndef rectangle_perimeter(l, b):\n\treturn 2...",53
32,Write a python function to find the area of a ...,\n\ndef findArea(r):\n\tPI = 3.142\n\treturn P...,52
...,...,...,...
4308,write a program which accepts basic mathematic...,expression = raw_input()\nprint(eval(expressio...,49
4312,generate a random float where the value is bet...,import random\nprint(random.random() * 100 - 5)\n,47
4321,write a program to shuffle and print the list ...,"from random import shuffle\nli = [3, 6, 7, 8]\...",67
4330,"With two given lists [1,3,6,78,35,55] and [12,...","set1 = set([1, 3, 6, 78, 35, 55])\nset2 = set(...",116


In [281]:
train_df['Code']=train_df['Code'].str.replace('\t', '<TAB>')
valid_df['Code']=valid_df['Code'].str.replace('\t', '<TAB>')

In [282]:
train_df[train_df['Code'].str.contains('\t')]

,Description,Code,len


In [283]:
valid_df[valid_df['Code'].str.contains('\t')]


,Description,Code,len


In [284]:
train_df = train_df[['Description', 'Code']]
valid_df = valid_df[['Description', 'Code']]

train_df.to_csv('train.tsv', index=False, sep='\t')
valid_df.to_csv('valid.tsv', index=False, sep='\t')

In [285]:
df_final1 = df_final1.drop(df_final1[df_final1['len'] == 0].index)

In [286]:
df_final2 = df_final1.sort_values(by='Description', key=lambda x:x.str.len())

In [287]:
df_final2.sort_values(by='len').iloc[100:200]

,Description,Code,len
3865,write a python function that removes element f...,"\n\ndef drop(a, n=1):\n\treturn a[n:]\n",34
3154,Write a Python function to return binary value...,def int_to_bin(a):\n\treturn bin(a)\n,34
1972,Write a python program to print a list with al...,\n\nlist1 = [5] * 10\n\n\nprint(list1)\n,34
2760,Write a python program to print a list with al...,\n\nlist1 = [5] * 10\n\n\nprint(list1)\n,34
1052,Write a Python function to return octal value ...,def int_to_oct(a):\n\treturn oct(a)\n,34
4261,Write a method which can calculate square valu...,def square(num):\n\treturn num ** 2\n,34
2216,write a python program to print squares of num...,for i in range(20):\n\tprint(i * i)\n,34
3551,write a python program to print squares of num...,for i in range(20):\n\tprint(i * i)\n,34
4266,Define a function that can convert a integer i...,def printValue(n):\n\tprint(str(n))\n,34
3156,Write a Python function to return hexadecimal ...,def int_to_hex(a):\n\treturn hex(a)\n,34


In [288]:
df_final2.iloc[:100]

,Description,Code,len
738,if else for,for i in range(5):\n\tif i == 1:\n prin...,103
743,Disassembler,import dis\n\n\ndef hello1_func():\n\ts = 'Hel...,78
1765,Split Strings,"word = ""Hello World""\nksplit = word.split(' ')...",82
2610,Split Strings,"word = ""Hello World""\nksplit = word.split(' ')...",82
732,random choice,"from random import choice\n\n\na, b, c = float...",118
735,Usge of *args,"def a_func(*args):\n\tprint('type of args:', t...",149
3951,Split Strings,"word = ""Hello World""\nksplit = word.split(' ')...",82
444,sort the list,words.sort()\n,13
695,args and sleep,import time\n\n\ndef report_arg(my_default=tim...,115
741,Usage of while,i = 0\nwhile i < 2:\n\tprint(i)\n\ti += 1\nels...,61
